In [38]:
realms = ['scourge_x2', 'legacy_x3', 'algalon_x4', 'sirus_x5', 'legacy_x10']

In [1]:
def get_url(page : int):
    return f'https://sirus.su/statistic/online?page={page}#/'


In [25]:
import os
import requests
from bs4 import BeautifulSoup

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0'
}

response = requests.get(get_url(1), headers=headers)
soup = BeautifulSoup(response.text, 'lxml')
count_of_pages = int(soup.find('ul', attrs={'class': 'pagination'}).find_all('li')[-2].text)
print(f'Pages: {count_of_pages}')

for i in range(count_of_pages):
    # time.sleep(1)
    if not os.path.exists(f'pages/index_page{i+1}.html'):
        response = requests.get(get_url(i+1), headers=headers)
        print(f'Downloaded page {i+1}')
        with open(f'pages/index_page{i+1}.html', 'w', encoding='utf-8') as f:
            f.write(response.text)

Pages: 353


## Создание Базы данных

In [43]:
import sqlite3

with sqlite3.connect('players.db') as db:
    cursor = db.cursor() 
    
    for realm in realms:
        print(f"""CREATE TABLE {realm}""")
        cursor.execute(f"""DELETE FROM {realm}""")
        cursor.execute(f"""CREATE TABLE IF NOT EXISTS {realm}
                   (id INTEGER PRIMARY KEY AUTOINCREMENT,
                   name TEXT,
                   level INTEGER,
                   class TEXT,
                   race TEXT)
                   """)
    db.commit()
    

CREATE TABLE scourge_x2
CREATE TABLE legacy_x3
CREATE TABLE algalon_x4
CREATE TABLE sirus_x5
CREATE TABLE legacy_x10


In [5]:
def insert_data(data, table_name):
    with sqlite3.connect('players.db') as db:
        cursor = db.cursor()
        cursor.execute(f"""INSERT INTO {table_name} (name, level, race, class) VALUES (?, ?, ?, ?)""", data)

In [6]:
def log(message, file='log.log'):
    with open(file, 'a', encoding='utf-8') as log:
        log.write(f'{message}\n')

In [44]:
from bs4 import BeautifulSoup
import lxml

played_id = 0
for i in range(count_of_pages):
    print(f'Page {i+1}/{count_of_pages}')
    try:
        with open(f'pages/index_page{i+1}.html', 'r', encoding='utf-8') as f:
            html = f.read()
    except Exception as e: 
        print('with exception', e)
        continue

    soup = BeautifulSoup(html, 'lxml')

    table = soup.find('table')

    trs = soup.find_all('tr')
    with sqlite3.connect('players.db') as db:
        cursor = db.cursor()
        for i, tr in enumerate(trs):
            if i == 0:
                continue
            tds = tr.find_all('td')
            name = tds[0].text.replace('\n', '').strip()
            level = tds[1].text.replace('\n', '').strip()
            race = tds[2].find('img', class_='race').get('title')
            class_ = tds[2].find('img', class_='class').get('title')
            realm = tds[3].text.replace('\n', '').strip().split("-")[0].lower().strip().replace(" ", "_")        
            
            log(
                message=f'#{played_id} {name} {level} {race} {class_} - {realm}', 
                file=f'logs/log_{realm}.log'
            )
            insert_data((name, level, race, class_), table_name=realm)
                
            played_id += 1
            
print(realms)

Page #1
Page #2
Page #3
Page #4
Page #5
Page #6
Page #7
Page #8
Page #9
Page #10
Page #11
Page #12
Page #13
Page #14
Page #15
Page #16
Page #17
Page #18
Page #19
Page #20
Page #21
Page #22
Page #23
Page #24
Page #25
Page #26
Page #27
Page #28
Page #29
Page #30
Page #31
Page #32
Page #33
Page #34
Page #35
Page #36
Page #37
Page #38
Page #39
Page #40
Page #41
Page #42
Page #43
Page #44
Page #45
Page #46
Page #47
Page #48
Page #49
Page #50
Page #51
Page #52
Page #53
Page #54
Page #55
Page #56
Page #57
Page #58
Page #59
Page #60
Page #61
Page #62
Page #63
Page #64
Page #65
Page #66
Page #67
Page #68
Page #69
Page #70
Page #71
Page #72
Page #73
Page #74
Page #75
Page #76
Page #77
Page #78
Page #79
Page #80
Page #81
Page #82
Page #83
Page #84
Page #85
Page #86
Page #87
Page #88
Page #89
Page #90
Page #91
Page #92
Page #93
Page #94
Page #95
Page #96
Page #97
Page #98
Page #99
Page #100
Page #101
Page #102
Page #103
Page #104
Page #105
Page #106
Page #107
Page #108
Page #109
Page #110
Page #11

In [53]:
import prettytable
import json
import csv

def print_table(table_name):
    with sqlite3.connect('players.db') as db:
        cursor = db.cursor()
        cursor.execute(f"""SELECT * FROM {table_name}""")
        table = prettytable.from_db_cursor(cursor=cursor)
    with open(f'tables/{table_name}.txt', 'w', encoding='utf-8') as f:
        f.write(str(table))
    # print(table)
def print_json(table_name):
    with open(f'json/{table_name}.json', 'w', encoding='utf-8') as j:
        with sqlite3.connect('players.db') as db:
            cursor = db.cursor()
            cursor.execute(f"""SELECT name, level, class, race FROM {table_name}""")
            json.dump(cursor.fetchall(), j, indent=4, ensure_ascii=False)

def print_csv(table_name):
    with open(f'csv/{table_name}.csv', 'w', encoding='utf-8', newline='') as c:
        with sqlite3.connect('players.db') as db:
            cursor = db.cursor()
            cursor.execute(f"""SELECT * FROM {table_name}""")
            writer = csv.writer(c, delimiter=';')
            
            writer.writerow
            (
                ('ID', 'Ник', 'Уровень', 'Раса', 'Класс')
            )
            
            writer.writerows(cursor.fetchall())

for realm in realms:
    print(f'TABLE {realm}')
    print_table(realm)
    print_json(realm)
    print_csv(realm)

TABLE scourge_x2
TABLE legacy_x3
TABLE algalon_x4
TABLE sirus_x5
TABLE legacy_x10
